In [5]:
import time
import numpy as np # Added numpy import for array manipulation
# This script simulates the core logic for Project 7: Scenario-Based Safety
# Validation, demonstrating test automation and safety reporting.

# --- Scenario Definitions ---

def define_scenario_j_walk():
    """
    Defines a critical safety scenario for testing (Pedestrian J-Walk).
    """
    scenario = {
        "name": "S_PED_J_WALK_001",
        "description": "Pedestrian suddenly enters the road from behind a parked obstacle.",
        "input_conditions": {
            "vehicle_speed_mph": 25,
            "distance_to_obstacle_ft": 80,
            "reaction_time_expected_s": 0.5
        },
        "acceptance_criteria": [
            "Collision must be avoided (Pass if Min Distance > 0 ft).",
            "Deceleration must not exceed 0.7g (Comfort/Safety limit)."
        ]
    }
    return scenario

# --- Dataset Loading Simulation ---

def load_scenario_metrics(num_runs=10):
    """
    Simulates loading a dataset (e.g., from a CSV or Simulation logs)
    containing recorded metrics for a specific scenario run.

    Returns a NumPy array where:
    Column 0: min_distance_ft (Safety Metric)
    Column 1: peak_deceleration_g (Comfort/Safety Metric)
    """
    # Simulate 10 test runs with mixed results:
    # 3x Pass, 3x Pass w/Warning (hard braking), 4x Fail (near collision/collision)
    data = np.array([
        [1.5, 0.4], # PASS (Safe distance, smooth stop)
        [3.2, 0.5], # PASS
        [0.8, 0.8], # PASS w/WARNING (Hard brake, but avoided collision)
        [0.05, 0.75], # PASS w/WARNING (Close call, hard brake)
        [2.1, 0.6], # PASS
        [-0.1, 0.5], # FAIL (Collision, distance <= 0)
        [0.5, 0.9], # FAIL (Hard brake AND close call/potential fail)
        [4.0, 0.3], # PASS
        [0.0, 0.65], # FAIL (Direct Collision)
        [1.1, 0.85] # PASS w/WARNING (Hard brake, but avoided collision)
    ])
    return data

# --- Test Execution Logic ---

def execute_scenario(scenario, metrics_data):
    """
    Processes the recorded metrics data against the scenario's acceptance criteria.
    """
    print(f"Executing Scenario: {scenario['name']} - {scenario['description']}")
    time.sleep(0.1) # Simulate quick processing time

    # --- Process Recorded System Output ---
    min_distance_to_pedestrian_ft = metrics_data[0]
    peak_deceleration_g = metrics_data[1]

    # --- Acceptance Criteria Evaluation ---

    test_result = "PASS"
    fail_reasons = []

    # Check 1: Collision Avoidance (CRITICAL SAFETY METRIC)
    if min_distance_to_pedestrian_ft <= 0.0:
        fail_reasons.append("Collision detected (Min Distance <= 0.0 ft)")
        test_result = "FAIL"

    # Check 2: Comfort/Safety Deceleration Limit (COMFORT/SOFT SAFETY METRIC)
    if peak_deceleration_g > 0.7:
        fail_reasons.append(f"Hard braking detected ({peak_deceleration_g:.2f}g > 0.7g limit)")
        # We allow a PASS if it avoided collision, but report the soft failure
        if test_result == "PASS":
            test_result = "PASS_WITH_WARNING"
        else:
            test_result = "FAIL" # Keeps the hard FAIL status if Check 1 failed

    return {
        "scenario_name": scenario['name'],
        "result": test_result,
        "metrics": {
            "min_distance_ft": min_distance_to_pedestrian_ft,
            "peak_deceleration_g": peak_deceleration_g
        },
        "fail_reasons": fail_reasons
    }

# --- Reporting ---

def generate_test_report(results):
    """
    Generates a structured report, a key deliverable for TPMs.
    """
    print("\n=======================================================")
    print("  SCENARIO SAFETY VALIDATION REPORT (Project 7)")
    print("=======================================================")

    total = len(results)
    passed = sum(1 for r in results if r['result'] == 'PASS')
    warned = sum(1 for r in results if r['result'] == 'PASS_WITH_WARNING')
    failed = sum(1 for r in results if r['result'] == 'FAIL')

    for i, r in enumerate(results):
        print(f"\n[RUN {i+1}: {r['result']}] {r['scenario_name']}:")
        print(f"  > Min Distance to Pedestrian: {r['metrics']['min_distance_ft']:.2f} ft")
        print(f"  > Peak Deceleration:          {r['metrics']['peak_deceleration_g']:.2f} g")
        if r['fail_reasons']:
            print("  > FAILURES/WARNINGS:")
            for reason in r['fail_reasons']:
                print(f"    - {reason}")

    print("\n-------------------------------------------------------")
    print(f"SUMMARY: Total Scenarios Run: {total}")
    print(f"         PASS: {passed}")
    print(f"         PASS (Warning): {warned}")
    print(f"         FAIL: {failed}")
    print("-------------------------------------------------------")
    if failed > 0:
        print("ACTION REQUIRED: Block deployment until critical failures are resolved.")


# --- Main Execution ---

if __name__ == "__main__":
    # Define the test scenario
    test_scenario = define_scenario_j_walk()

    # Load the dataset of recorded metrics (10 runs)
    scenario_dataset = load_scenario_metrics(num_runs=10)

    # Run the validation for every recorded run in the dataset
    test_results = []
    print(f"Loaded dataset with {len(scenario_dataset)} recorded runs to validate.")

    for run_metrics in scenario_dataset:
        test_results.append(execute_scenario(test_scenario, run_metrics))

    generate_test_report(test_results)

Loaded dataset with 10 recorded runs to validate.
Executing Scenario: S_PED_J_WALK_001 - Pedestrian suddenly enters the road from behind a parked obstacle.
Executing Scenario: S_PED_J_WALK_001 - Pedestrian suddenly enters the road from behind a parked obstacle.
Executing Scenario: S_PED_J_WALK_001 - Pedestrian suddenly enters the road from behind a parked obstacle.
Executing Scenario: S_PED_J_WALK_001 - Pedestrian suddenly enters the road from behind a parked obstacle.
Executing Scenario: S_PED_J_WALK_001 - Pedestrian suddenly enters the road from behind a parked obstacle.
Executing Scenario: S_PED_J_WALK_001 - Pedestrian suddenly enters the road from behind a parked obstacle.
Executing Scenario: S_PED_J_WALK_001 - Pedestrian suddenly enters the road from behind a parked obstacle.
Executing Scenario: S_PED_J_WALK_001 - Pedestrian suddenly enters the road from behind a parked obstacle.
Executing Scenario: S_PED_J_WALK_001 - Pedestrian suddenly enters the road from behind a parked obstac